In [1]:
import pandas as pd
import requests
from os import listdir
from os.path import isfile

jonhs_hopkins_github_url = 'https://api.github.com/repos/CSSEGISandData/COVID-19/contents/csse_covid_19_data/csse_covid_19_daily_reports'

In [2]:
response = requests.get(jonhs_hopkins_github_url)

if response:
    github_content_files = response.json()

In [3]:
def standardization(file):
  df_standard = pd.DataFrame()
  df = pd.read_csv(file, sep=',', encoding='utf-8')
        
  standard_fields = []
  standard_fields.append(''.join(df.columns[df.columns.str.contains('Province')].format()))
  standard_fields.append(''.join(df.columns[df.columns.str.contains('Country')].format()))
  standard_fields.append(''.join(df.columns[df.columns.str.contains('Update')].format()))
  standard_fields.append(''.join(df.columns[df.columns.str.contains('Confirmed')].format()))
  standard_fields.append(''.join(df.columns[df.columns.str.contains('Deaths')].format()))
  standard_fields.append(''.join(df.columns[df.columns.str.contains('Recovered')].format()))
  standard_fields.append(''.join(df.columns[df.columns.str.contains('Lat')].format()))
  standard_fields.append(''.join(df.columns[df.columns.str.contains('Long')].format()))                                                          
  standard_fields = [string for string in standard_fields if string != ""]
  
  if len(standard_fields) != len(df.columns):
    df_standard = df[standard_fields]
    df_standard.columns = ['Province_State', 'Country_Region', 'Last_Update', 'Confirmed', 'Deaths', 'Recovered', 'Lat', 'Long']
    df_standard['file'] = file
  
  return df_standard

In [5]:
for file in github_content_files:
    if '.csv' in file['download_url']:
        r = requests.get(file['download_url'], allow_redirects=True)
        open(file['name'], 'wb').write(r.content)

In [6]:
df_temp = pd.DataFrame()
df = pd.DataFrame()

onlyfiles = [f for f in listdir() if isfile(f)]
for file in onlyfiles: 
  df_temp = standardization(file)
  if not df_temp.empty:
    df = pd.concat([df,df_temp])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
df.to_csv('consolidated_covid_19_data.csv', sep=',',encoding='utf-8')

In [9]:
df.tail()

,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Lat,Long,file
3158,NaN,West Bank and Gaza,2020-04-29 02:32:29,343,2,71,31.952200,35.233200,04-28-2020.csv
3159,NaN,Western Sahara,2020-04-29 02:32:29,6,0,5,24.215500,-12.885800,04-28-2020.csv
3160,NaN,Yemen,2020-04-29 02:32:29,1,0,1,15.552727,48.516388,04-28-2020.csv
3161,NaN,Zambia,2020-04-29 02:32:29,95,3,42,-13.133897,27.849332,04-28-2020.csv
3162,NaN,Zimbabwe,2020-04-29 02:32:29,32,4,5,-19.015438,29.154857,04-28-2020.csv


In [10]:
df.shape

(425513, 9)